# Neural Network Regression task - Bike sharing

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return has become automatic. Through these systems, a user is able to easily rent a bike from a particular position and return it at another place.

The dataset contains the hourly count of rental bikes between years 2011 and 2012 in the Capital Bikeshare system (Washington DC) with the corresponding weather and seasonal information.

The goal of this task is to train a regressor to predict total counts of bike rentals based on the provided features for a given hour.

## Data source
[http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

## Feature description
* **dteday** - date time stamot
* **season** - season (1: spring, 2: summer, 3: fall, 4: winter)
* **yr** - year (0: 2011, 1: 2012)
* **mnth** - month (1 to 12)
* **hr** - hour (0 to 23)
* **holiday** - 1 if the day is a holiday, else 0 (extracted from [holiday schedules](https://dchr.dc.gov/page/holiday-schedules))
* **weekday** - day of the week (0 to 6)
* **workingday** - is 1 if day is neither weekend nor holiday, else 0.
* **weathersit**
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp** - Normalized temperature in degrees of Celsius.
* **atemp** - Normalized feeling temperature in degrees Celsius.
* **hum** - Normalized relative humidity.
* **windspeed** - Normalized wind speed.
* **casual** - Count of casual users.
* **registered** - Count of registered users.
* **cnt** -  Count of total rental bikes including both casual and registered. This is the target value.

In [ ]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/mlcollege/introduction-to-ml/master/data/bikes.csv', sep=',')
data.head()

## Add some features from the past

Since we have time stamp of every measurement, we can see the data as a time series and use data from the past. We add one feature column computed from the data of the previous hour.

In [ ]:
# Add features from the past: create lagged features based on previous hour
data.sort_values(['dteday', 'hr'])
cnt = data['cnt']
data['hist'] = cnt.shift(1)
data = data[1:]
data.head()

## Neural Network regressor

Implement a neural network regressor based on all reasonable features from the input data set. Notice that some of the features from the input data cannot be used.

### Data preparation

Prepare train and test data sets.

In [ ]:
from sklearn.model_selection import train_test_split

# Prepare the data including the new 'hist' feature
X_all = data[['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit','temp', 'atemp', 'hum', 'windspeed', 'hist']]
y_all = data['cnt']

# Split data into training (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(
    X_all,
    y_all,
    random_state=1,
    test_size=0.2)

print('Train size: {}'.format(len(X_train)))
print('Test size: {}'.format(len(X_test)))

Standardize the features

In [ ]:
# Standardize features: fit scaler on training data and apply to both train and test
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Training a regressor
Design and train a regression model. Use the [mean squared error](https://keras.io/losses/) loss function. Experiment with various architectures, [activation functions](https://keras.io/activations/) and [optimizers](https://keras.io/optimizers/).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

# Design a neural network for regression
model = Sequential()

# Add first hidden layer with tanh activation
model.add(Dense(32))
model.add(Activation('tanh'))
#model.add(Dropout(0.5))

# Add second hidden layer with tanh activation
model.add(Dense(32))
model.add(Activation('tanh'))
#model.add(Dropout(0.5))

# Add output layer with relu activation (suitable for regression)
model.add(Dense(1))
model.add(Activation('relu'))

Compile the model

In [ ]:
# Compile the model with mean squared error loss and adam optimizer
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae'])

Train the model

In [ ]:
# Train the model with validation on test set
model.fit(X_train, y_train,
          batch_size = 128, epochs = 500, verbose=1,
          validation_data=(X_test, y_test))

### Evaluate the models

Measure mean squared error and mean absolute error evaluation metrics on both train and test data sets. Compute the mean and standard deviation of the target values.

In [ ]:
# Evaluate the model on the test set
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred = model.predict(X_test)
print ("Test mean: {}, std: {}".format(np.mean(y_test), np.std(y_test)))
print("Test Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("Test Mean absolute error: {:.2f}".format(mean_absolute_error(y_test, y_pred)))

In [ ]:
# Evaluate the model on the training set
y_pred = model.predict(X_train)
print("Train Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_train, y_pred))))
print("Train Mean absolute error: {:.2f}".format(mean_absolute_error(y_train, y_pred)))